# Q-learning with neural networks

In [1]:
# a few packages we need to import
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython 

import torch

import pendulum

The goal of this homework is to implement the Q-learning with a neural network for the Q function to solve the inverted pendulum problem.

<img src='pendulum.png' width="120">

In the following, we write $x = \begin{pmatrix} \theta \\ \dot{\theta} \end{pmatrix}$ as the vector of states of the system.

## System dynamics
* The system dynamics is implemented in the `pendulum.py` function. The dynamics is implemented in `pendulum.step`.
* The allowed control inputs are $[-5,0,5]$

## Cost function
The goal is to find a policy that minimizes the following cost
$$\min \sum_{n=0}^N \alpha^n g(x,u)$$
where
$$g(x,v,u) = 0.01*(1-\cos(x-\pi))^2 + 0.001* v^2 + 0.00001*u^2$$
which gives a high cost for states far from $\pi$ (i.e. far from the inverted position) or states with non zero velocity or high controls



## Q-learning algorithm to implement
For each episode:
* Initialize the episode $x_0 = [0,0]$
* For each step of the episode:
    * Select $u_n$ using an $\epsilon$-greedy policy
    * Compute the next state $x_{n+1}$
    * Compute the target $y_n = g(x_n,u_n) + \alpha \min_a Q(x_{n+1},a)$
    * Do one SGD step on the neural network parameters to minimize $(Q(x,u) - y_t)^2$


## Parameters:
* Episode length 100 steps
* Discount factor $\alpha = 0,99$
* Learning rate (for SGD) 0.1
* $\epsilon = 0.1$



## Using PyTorch
You need to install and use PyTorch for the neural network and do the optimization. 

You may want to use the following functions:
* [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)
* [`torch.nn.MSELoss`](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html)

The neural network is given below

In [2]:
## we define the neural network to be used for Q-learning
## 2 hidden layers with 64 nodes
## 2 inputs (state)
## 3 outputs for the 3 possible controls
D_in, H, D_out = 2, 64, 3

q_function = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)

## we initialize the network parameters to 0
for params in q_function.parameters():
    params = torch.zeros_like(params)


### possible controls
possible_controls = np.array([-5.,0.,5.])

# Questions:
1. Implement the Q-learning algorithm described above
2. Test that it works with and without pushes using the code below
3. Plot the cost per episode (to visualize learning)
4. Plot the learned value function (in 2D as a function of pendulum position and velocity) as well as the policy.
5. Describe the algorithm and put the plots in a short report (max 2 pages) and include a video of the pendulum.

## Testing
You can test your results with the code below which use the Q-function to create a controller send to the `animate_robot` function.
You can choose to save also a movie of the animation and toggle the animation with a disturbance.

In [4]:
x0 = np.zeros((2,1))
def controller(x):
    u_pred = torch.argmin(q_function(torch.as_tensor(x, dtype=torch.float).unsqueeze(0))).item()
    u = possible_controls[u_pred]
    return u
    
pendulum.animate_robot(x0,controller,push=False, save_movie=True)


<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAG5ptZGF0AAACrwYF//+r
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2NCByMzEwOCAzMWUxOWY5IC0gSC4yNjQvTVBF
Ry00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMyAtIGh0dHA6Ly93d3cudmlkZW9sYW4u
b3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFs
eXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVk
X3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBk
ZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE5
IGxvb2thaGVhZF90aHJlYWRzPTMgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50
ZXJsYWNlZD0wIGJsdXJheV9jb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBi
X3B5cmFtaWQ9MiBiX2FkYXB0PTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29w
PTAgd2VpZ2h0cD0yIGtleWludD0yNTAga2V5aW50X21pbj0xMCBzY2VuZWN1dD00MCBpbnRyYV9y
ZWZyZXNoPTAgcmNfbG9va2FoZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0w
LjYwIHFwbWluPTAgcXBtYXg9NjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAA
DSxliIQAEf/+94gfMstp+TrXchHnrS6tH1DuRnFepL3+IAAAAwAAAwACHHgr8h1jA36boAAANaAG
nDnk/0uKTTxANBMklOx5AfoyzTNjGZoxSMjI2XqpG5muyFpkpA6oCAPVnxylzDLYhuV6SiLysV19
ZrkfNVeuuW3Q9PnAIUDhMZl7zUCMeOI9XyhO34h2N6JqaLaBMEJBECZ+5uHUUpUwEyHMK0SddgK8
GQ+8N9YKy9+mi5xcrkYkRWJQER7+UzLKhVSfJl1dCKS9cKV2QpOCIcN2cB6dc7SsaFnK+lNL704K
VLG608M5FPwh6EL0EBwAfACHU1dX1oJZzgeYkF9jIDc1B3hLlj82wiQfYvK10RzuN9Q1U62r1Osc
ElerAZGzgImHEQ9AUCDlSjLBCymJ0ZW3j+UdffVRQ4cAzpTgLL/sjPAGF6YX+yqMIy2LXBh9gX6P
dyg9TIX2+Qqsog+WDM8kjQI5D1HDegmYnuqXpe/MBsNnkgpFFgiOvUE347T2ajp0qdYwaDMgqlYf
58zP9aYJ3tJmPvGaMh2VDhGmbYiXwFVLHq36Opwl7f12x7Sxb1T9184FQlIt/RBporGW/9z2nx2y
oEwSMv63fDm8c5dmi2IKv5xjdW4tSBSTS+kLL5X5/FQkLhX9ckakEaSgYmb4KFzr+7zD2pktr220
9d3VyuJMbIx0+qpYJDVQjrgN0Wti9ZNxLnWUCvXYDf0ZvAJQZU1iwCen8eYbjTyQ78hjP2OMtxQ6
rhx4MRvCG0CM9lrLuUAAAAMAAAMAAAMAA4vC7v82dVh/Yjf5u6X9pKY2D1vSU0FyAvT9C0i2lOu5
6d10+xKTprmLCJQQx23WAdIqDMSBktDH4Q6IoXbbibqSoFfHlQS8hJvY4sYuZ+gcP0Z7klP+frIq
94jQAdPEr9QpOUL3t+oRw8ZzCpAnQDVPU0xLy1EylDTmnpUkPlf8m2WgRe/srmsI+wizvgfknC2J
VyiyezagM6t6r6yvnCZlGgNZo8k9/vYUqjLvlvYNseXk/U6o/WtRML80H01/7OIlGxztIEApdHzU
A21BEJhD97z0nqKjtaz9E21ifhLF4p3gwKA4rm0+liGZsTQkEP+wdDciC/4bfX06KaTguiNbM9DL
Qg/6nq4APWG6vXw/qhQ09ntOzR2fY7BmnMbU9vJ8wGWJ9HSYi3ccBJ+NT5hPRpr/RGbkeTiRFK1J
ASKRA2uGZ7Vjb8AAAAMAAAMAANK3cz/AeM1zm0O1T6opP6vJxy5netqwLbmzOPJp+iCSOc4y3MSj
W3gLXUBv3R5BTQt5dmFpKUJhgIlmf7MA1tTAB79p3P8xMZbGtu1iXWSNvuJOwtiKB3pPFOe8x6FE
LG2nW4Vxuulk2bLhy9HpBcJlwzTQo+ER/vLUhs+6nBb6xrnPGgQpvKbqQBTv9iSNsgYzbhO/JwW8
B3UfiuZ9iF3UCuzDYVwE3DsC1mZR7kOcB+a9eRCXUb+F2KqpVC/8TP9Amny6wGJEuKHlIhnhDPEz
Cf763neu+No6j5G3SvEXQQ7PwQY1T2gBfnDaPe9MzT8lojWRIhJsF6OW+QXU816ah95Iuzqr1f10
lSzSn61vKYssX5F61sK7wPUbFn5/3/9FvS1pt7iulOXjVBOIH4CeJWEXmLLDdaiiZOjs+3kaFekH
y3ePF9yVRFLFRCfQFPeOIMepAmXeqQH8kKVdgBzBEH5e8AJE/zoYVtuERzNfVSTKiX/Hgu0ZgGT5
Y7TV1qW3ADNM7QKcH3v4uzYo+TMEq5I0ydbUL4UIzer5Ky4qG+P/wEaGuCrqrZsOdr6Vrakx3Q6x
EXNK3thnIWqTWAcCUmJ5xDdzEzkb8nEVDiH+JsZmwjSzAFtFfExh4+RpKT/8+W5fzdKsu4AIYgrJ
WMl0YkhiMXkX2DB+xUkpciixsBvYN0OVyOUx3luHvCwQ8TLU6aj3GWMM8Zwm71u6m+zAlv7q95Vo
fTGBeXRs0LjD4Eqa8qa75y52akqNMNKt6qrgh6cwjUxcuYoWOrEWPbb/IQ2RbQRVin9ABCDfKxVm
qDV0KEtF29Vs+af+3s8NF/v+FZ5ZPJTGlejjyQepi8SYclMxxkQP23Wqs9Hk/gKD+8bV6C0CXsyJ
dyD8vd8KGpVW7ADUEXrYSbG4Mw56rXY9JlONSUSVN/6Ndu2M67dvRmhXGXupjIKtGCa313uaukgw
fOV12EShPEQYjploAQHQnkALktgMzl51mXXf/9IAfEKt/5bv6o9S+ePrqNGlwumFt2x2pZLjxAcj
APkMmZiVp6qdypjn7PZIVeqdTz75KFUMadq95UZKhvW2MY7ZbeNmJ2tdPWP99h5yDMq7NW+VD18K
RDoAzdyA+QvMgl0iU2mDRh90QbA4CUtIxrhmcivrGMxkOxyGG9ZMMNQAE957AdXz0Hnr4NKzmo1B
BNiDYEvGOYNj3M762j80wSAZ5SBX2cLRngvpUoeXUc3g9l2NO7UxFAMWkCNWlQgwnLC4hQOIw9ym
lYu//hgvxzmb8NNP1junJRKbwftmGvakbYSr2oJEc9zL6mypWUwgQfKm3/tdsoeaqXA57ZSCFVlQ
FBMHRtsEpqkAKpx0L4hKY+x/Aq2Lct/d50EF4j6aafHIJ3qIL+f4w3QNyRoP+9pRUpf8gzAWtyMQ
rnbOj/6Ts9++vXS14lj9QR1sMuG9QITe79U5odyrN1A+oXsim7dWpExi2Sxoz5A+YwEphtvkanX9
q445LFC3gSKNOsIcUnau3ZxPV3Zgh5Iufe9xhuFK8zI2pmcJ1dOx3Z6su2nz/9sMjm9u8yvqmv9r
B++CpPY/wsGptkNmXQ0dmOOD+Mt4VichmsElv97TI58AsCKTm5Tyj4BIdaUEYTG0flkBqQ3jtAG6
ss0u8nHA4Kv1Wq6m7S+TE